In [2]:
from owlready2 import *
from fuzzywuzzy import fuzz
import numpy as np

C:\Users\Baraa\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
from os import listdir
from os.path import isfile, join
currentPath = join(os.path.abspath(os.getcwd()), 'ontologies')

ontologiesFiles = [f for f in listdir(join(currentPath)) if isfile(join(currentPath, f))]

In [6]:
basePath = f"file://{currentPath}"

paths = [join(basePath, filename) for filename in ontologiesFiles]
ontologies = [get_ontology(path).load() for path in paths]

In [7]:
def getClass(onto, name):
    ratioArr = [fuzz.ratio(name, c.name) for c in onto.classes()]
    index = np.argmax(ratioArr)
    name = list(onto.classes())[index].name

    return name if ratioArr[index] > 60 else None


def getPropertyName(onto, name):
    ratioArr = [fuzz.ratio(name, c.name) for c in onto.properties()]
    index = np.argmax(ratioArr)
    name = list(onto.properties())[index].name

    return name if ratioArr[index] > 60 else None
    
    
def getProperty(obj, onto, name):
    return getattr(obj, getPropertyName(onto, name))

def propertyExist(obj, onto, name): 
    property = getPropertyName(onto, name)
    return (not property is None) and len(getattr(obj, property)) > 0


def classExist(onto, name):
    className = getClass(onto, name)
    return not className is None and not onto[className] is None

In [90]:
className = 'onto34'
unionOnto = get_ontology(f"http://test.org/{className}.owl")

with unionOnto:
    class Algorithm(Thing):
        pass
        
    class useAlgorithm(Algorithm >> Algorithm):
        pass
        
    class Task(Thing):
        pass
    
    class uses(Task >> Algorithm):
        pass
    
    class hasExample(Algorithm >> Task):
        pass
    
    class TaskInput(Thing):
        pass
    
    class hasInput(Task >> TaskInput):
        pass
        
    class DataSet(Thing):
        pass
    
    class canGetData(TaskInput >> DataSet):
        pass
    
    class Feature(Thing):
        pass
    
    class canExtract(TaskInput >> Feature):
        pass
    
    class TaskOutput(Thing):
        pass
    
    class produce(Task >> TaskOutput):
        pass
    
    class EvaluationMatric(Thing):
        pass
    
    class evaluatedBy(TaskOutput >> EvaluationMatric):
        pass
    
    class Limitation(Thing):
        pass
    
    class Parameter(Thing):
        pass
    
    class sufferFrom(Algorithm >> Limitation):
        pass
    
    class solvedBy(Limitation >> Algorithm):
        pass
    
    class learns(Algorithm >> Parameter):
        pass
    
    class learns(Parameter >> Algorithm):
        pass
        inverse_property = learns
    
    class Layer(Thing):
        pass
    
    class hasLayer(Algorithm >> Layer):
        pass
    
    class isPartOf(Layer >> Algorithm):
        pass
    
    class HyperParameter(Thing):
        pass
    
    class hasHyperParameter(Algorithm >> HyperParameter):
        pass
    
    class Function(Thing):
        pass
    
    class useFunction(Algorithm >> Function):
        pass
    
    class Model(Thing):
        pass
    
    class hasModel(Algorithm >> Model):
        pass
    
    class OptimizationAlgorithm(Thing):
        pass
    
    class useOptimizer(Algorithm >> OptimizationAlgorithm):
        pass
    
    class Operator(Thing):
        pass
    
    class execute(Algorithm >> Operator):
        pass
    
    class label(DataProperty):
        range = [str]
    
    
    
        
    for onto in ontologies:
        if classExist(onto, 'Algorithm'):
            for algo in onto.search(type = onto[getClass(onto, 'Algorithm')]):
                
                algorithm = Algorithm(algo.name)
                setattr(algorithm, 'label', [algo.name])

                if propertyExist(algo, onto, 'useAlgorithm'):
                    algorithm.useAlgorithm = [Algorithm(a.name) for a in getProperty(algo, onto, 'useAlgorithm')]
                    
                if propertyExist(algo, onto, 'hasExample'):
                    algorithm.hasExample = [Task(a.name) for a in getProperty(algo, onto, 'hasExample')] 
                    
                if propertyExist(algo, onto, 'learns'):
                    algorithm.learns = [Parameter(a.name) for a in getProperty(algo, onto, 'learns')] 
                    
                if propertyExist(algo, onto, 'sufferFrom'):
                    algorithm.sufferFrom = [Limitation(a.name) for a in getProperty(algo, onto, 'sufferFrom')] 
                    
                if propertyExist(algo, onto, 'hasLayer'):
                    algorithm.hasLayer = [Layer(a.name) for a in getProperty(algo, onto, 'hasLayer')] 
                    
                if propertyExist(algo, onto, 'hasHyperParameter'):
                    algorithm.hasHyperParameter = [HyperParameter(a.name) for a in getProperty(algo, onto, 'hasHyperParameter')] 
                    
                if propertyExist(algo, onto, 'useFunction'):
                    algorithm.useFunction = [Function(a.name) for a in getProperty(algo, onto, 'useFunction')] 
                    
                if propertyExist(algo, onto, 'hasModel'):
                    algorithm.hasModel = [Model(a.name) for a in getProperty(algo, onto, 'hasModel')] 
                    
                if propertyExist(algo, onto, 'useOptimizer'):
                    algorithm.useOptimizer = [OptimizationAlgorithm(a.name) for a in getProperty(algo, onto, 'useOptimizer')] 
                
                if propertyExist(algo, onto, 'execute'):
                    algorithm.execute = [Operator(a.name) for a in getProperty(algo, onto, 'execute')] 
                    
        
        if classExist(onto, 'Task'):   
            for task in onto.search(type = onto[getClass(onto, 'Task')]):
                t = Task(task.name)
                setattr(t, 'label', [task.name])
                
                
                if propertyExist(task, onto, 'hasInput'):
                    t.hasInput = [TaskInput(a.name) for a in getProperty(task, onto, 'hasInput')]
                
                if propertyExist(task, onto, 'uses'):
                    t.uses = [Algorithm(a.name) for a in getProperty(task, onto, 'uses')]
                    
                if propertyExist(task, onto, 'produce'):
                    t.produce = [TaskOutput(a.name) for a in getProperty(task, onto, 'produce')]
        
        if classExist(onto, 'Input'):   
            for input in onto.search(type = onto[getClass(onto, 'Input')]):
                i = TaskInput(input.name)
                setattr(i, 'label', [input.name])
                
                if propertyExist(input, onto, 'canGetData'):
                    t.canGetData = [DataSet(a.name) for a in getProperty(input, onto, 'canGetData')]
                    
                if propertyExist(input, onto, 'canExtract'):
                    t.canExtract = [Feature(a.name) for a in getProperty(input, onto, 'canExtract')]
                    
        if classExist(onto, 'Dataset'):   
            for data in onto.search(type = onto[getClass(onto, 'Dataset')]):
                if data.name != 'DataSet':
                    d = DataSet(data.name)
                    setattr(d, 'label', [data.name])
                    
        if classExist(onto, 'Feature'):   
            for feature in onto.search(type = onto[getClass(onto, 'Feature')]):
                f = Feature(feature.name)
                setattr(f, 'label', [feature.name])
                
        if classExist(onto, 'Output'):   
            for output in onto.search(type = onto[getClass(onto, 'Output')]):
                o = TaskOutput(output.name)
                setattr(o, 'label', [output.name])
                
                if propertyExist(output, onto, 'evaluatedBy'):
                    o.evaluatedBy = [EvaluationMatric(a.name) for a in getProperty(output, onto, 'evaluatedBy')]
                
        if classExist(onto, 'EvaluationMatric'):   
            for em in onto.search(type = onto[getClass(onto, 'EvaluationMatric')]):
                e = EvaluationMatric(em.name)
                setattr(e, 'label', [em.name])
                
        if classExist(onto, 'Parameter'):   
            for parameter in onto.search(type = onto[getClass(onto, 'Parameter')]):
                p = Parameter(parameter.name)
                setattr(p, 'label', [parameter.name])
                
                if propertyExist(parameter, onto, 'learns'):
                    p.learns = [Algorithm(a.name) for a in getProperty(parameter, onto, 'learns')]
                    
        if classExist(onto, 'Limitation'):   
            for limitation in onto.search(type = onto[getClass(onto, 'Limitation')]):
                l = Limitation(limitation.name)
                setattr(l, 'label', [limitation.name])
                
                if propertyExist(limitation, onto, 'solvedBy'):
                    l.solvedBy = [Algorithm(a.name) for a in getProperty(limitation, onto, 'solvedBy')]
                    
        if classExist(onto, 'Layer'):   
            for layer in onto.search(type = onto[getClass(onto, 'Layer')]):
                if layer.name != 'Layer':
                    p = Layer(layer.name)
                    setattr(p, 'label', [layer.name])

                    if propertyExist(layer, onto, 'isPartOf'):
                        p.isPartOf = [Algorithm(a.name) for a in getProperty(layer, onto, 'isPartOf')]
                    

In [91]:
unionOnto.save(file = "test19.owl")